In [8]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vaini,TO,2021-09-25 14:55:42,-21.2000,-175.2000,69.96,78,75,8.05
1,1,Gushikawa,JP,2021-09-25 14:55:42,26.3544,127.8686,83.43,94,1,6.91
2,2,Arlit,NE,2021-09-25 14:55:42,18.7369,7.3853,103.10,6,2,8.52
3,3,Mataura,NZ,2021-09-25 14:54:13,-46.1927,168.8643,39.47,98,90,4.32
4,4,Punta Arenas,CL,2021-09-25 14:52:24,-53.1500,-70.9167,41.11,65,75,17.27


In [9]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [10]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [11]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [12]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Gushikawa,JP,2021-09-25 14:55:42,26.3544,127.8686,83.43,94,1,6.91
7,7,Bata,GQ,2021-09-25 14:55:43,1.8639,9.7658,79.27,78,98,7.63
10,10,Bredasdorp,ZA,2021-09-25 14:54:55,-34.5322,20.0403,81.72,18,97,24.16
17,17,Sao Filipe,CV,2021-09-25 14:55:45,14.8961,-24.4956,78.80,75,49,6.98
27,27,Bacolod,PH,2021-09-25 14:55:46,10.6667,122.9500,79.11,83,97,6.13
30,30,Lac,AL,2021-09-25 14:55:47,41.6356,19.7131,77.11,49,0,4.36
32,32,Huilong,CN,2021-09-25 14:55:47,31.8111,121.6550,76.93,80,53,11.14
33,33,Champerico,GT,2021-09-25 14:55:48,14.3000,-91.9167,88.72,73,100,2.42
39,39,Avarua,CK,2021-09-25 14:51:41,-21.2078,-159.7750,75.25,69,75,11.50
42,42,Biak,ID,2021-09-25 14:54:58,-0.9131,122.8766,75.43,85,99,6.51


In [16]:
preferred_cities_df.count()

City_ID       212
City          212
Country       212
Date          212
Lat           212
Lng           212
Max Temp      212
Humidity      212
Cloudiness    212
Wind Speed    212
dtype: int64

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Gushikawa,JP,83.43,26.3544,127.8686,
7,Bata,GQ,79.27,1.8639,9.7658,
10,Bredasdorp,ZA,81.72,-34.5322,20.0403,
17,Sao Filipe,CV,78.80,14.8961,-24.4956,
27,Bacolod,PH,79.11,10.6667,122.9500,
30,Lac,AL,77.11,41.6356,19.7131,
32,Huilong,CN,76.93,31.8111,121.6550,
33,Champerico,GT,88.72,14.3000,-91.9167,
39,Avarua,CK,75.25,-21.2078,-159.7750,
42,Biak,ID,75.43,-0.9131,122.8766,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [20]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEC0og0xFHx06tGDN2CZYZd_10UFOY1egyJYc8JsyLz3MxwxVgxJEk2t3ybnwCJFEJHzLnMJAEH4reY4KXuaJx-67gqXrBffoE3BGByfWYQUYPSxwGaISHXLB5zvoSK7Nw3UO_cfKSFgldUZznSgohP82TPKMPXGW4bDXDQ2Q4FQU01vYBCy47Tz061yjpm6wMa0Ot-Bac3lAOjSTYXpB9PAXjiqZi-tdjZ_VL4Mr-Yyl8l_TwYTnVrjznoV3FKcJwAyvpNK1E2q6cpp_11SYL56w5SnZnHUo0uxCqD-RUL98hm_lM5qw2Gp6QOMC8WZSNYSHZ9m0FparP3ELyOSQyWNcBnAhyGMEONBe05rlYXh5Y3VAWN4CFCM_hv5sejSHNz5YW2_VrL1SWbxyqlnpb_7FSOlRs9y-jOXBWt6ePuQsXA0lqqLD-MB',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [21]:
len(hotels["results"])

20

In [31]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [39]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [44]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))